In [35]:
import pandas as pd
import numpy as np


import datetime
from datetime import timedelta

import os.path
import win32com.client as win32
from pathlib import Path

from IPython.core.display import HTML

In [36]:
# Download file from Outlook
outlook = win32.Dispatch('Outlook.Application').GetNameSpace('MAPI')
folder = outlook.Folders['vince.chau@outlook.com'].Folders['Inbox']

In [37]:
tdate = datetime.datetime.today()
strtoday = tdate.strftime('%Y%m%d')
strtoday

'20200503'

In [47]:
# List of emails in folder 
# Save it down
# Note this does not overwrite the existing file - You need to delete the existing file if re-run
for i in folder.Items:
    received = i.ReceivedTime
    subject = i.subject
    num_attachments = i.attachments.count
    strdate = received.strftime('%Y%m%d')
    
    if ('Test' in subject) and (strdate == strtoday):
        for attachment in i.attachments:
            filename = strdate + "_" + str(attachment)
            print(filename)
            def strRight(s, amount):
                return s[-amount:]
            if strRight(filename,4) == 'xlsx' and 'data' in filename:
                #attachment.saveasfile(r"C:\\" + filename) 
                #r"\\networkdrive.net\sharedrives\global\t0_flash\US\\"
                # if you do not specify a path: C:\Users\V\Documents\
                attachment.saveasfile(r"C:\A\Projects\Python\\" + filename)
        break

20200503_test_data.xlsx


In [48]:
# Create dates
start_date = datetime.date.today() + datetime.timedelta(-1)
end_date = datetime.date.today()

start_date

datetime.date(2020, 5, 2)

In [58]:
# Date Range
daterange=[]
day_count = (end_date - start_date).days + 1
for single_date in [d for d in (start_date + timedelta(n) for n in range(day_count)) if d<= end_date]:
    strdate = single_date.strftime('%Y%m%d')
    daterange.append(strdate)

strdate # '20200503'
daterange # ['20200502', '20200503']

['20200502', '20200503']

In [59]:
# Path where you saved the files
saved_dir = Path(r"C:\A\Projects\Python\\")
saved_files = list(saved_dir.glob("*.xlsx"))
saved_files

[WindowsPath('C:/A/Projects/Python/20200503_test_data.xlsx')]

In [61]:
# Create dataframes and load files
df_data = pd.DataFrame()
sample_data=[]

for file in saved_files:
    # get the date from the file name
    filename = os.path.split(file)
    fdate = filename[1].split("_")[0]
    
    if fdate == strdate:
        print('Import: ' + str(filename))
        xls = pd.ExcelFile(file)
        dfxls = pd.ExcelFile(file)
        
        # Import sheet with name "Sheet3"
        sample_data = pd.read_excel(dfxls, 'Sheet3')
        # Looks for column COB_Date
        sample_data['COB_Date'] = fdate
        df_data = df_data.append(sample_data, sort=True)
        pnlcol = [col for col in df_data.columns if 'P&L' in col or 'PnL' in col or 'PL' in col or 'pnl' in col]
        # Find PnL Column
        #i_pnl = df_data.columns.get_loc(pnlcol)
        df_data = df_data.rename(columns={pnlcol[0]:'PnL'})
        # Add classification
        df_data['Product'] = 'Equities'
        # Rename: uppercase everything
        df_data = df_data.rename(columns = lambda x: str(x).upper())
        # Rename: columns
        df_data = df_data.rename(columns={'PNL': 'PROFIT_LOSS', 'POSITION': 'EXPOSURE'})
        # Clean the dataset from any rows with Total in the Exposure column
        df_data = df_data[~df_data['EXPOSURE'].str.contains('Total',na=False)]
        
        # choose top positions
        desk_by_ticker = df_data.groupby(['EXPOSURE'],as_index=False).sum()
        desk_by_ticker['DeskName'] = 'Desk'
        top5 = desk_by_ticker.nlargest(5,'PROFIT_LOSS')
        
        # choose bottom positions
        desk_by_ticker = df_data.groupby(['EXPOSURE'],as_index=False).sum()
        desk_by_ticker['DeskName'] = 'Desk'
        top5 = desk_by_ticker.nsmallest(5,'PROFIT_LOSS')
        
df_data

Import: ('C:\\A\\Projects\\Python', '20200503_test_data.xlsx')


,EXPOSURE,MTM_T,MTM_T_1,PROFIT_LOSS,DeskName
4,Heineken,0.0,0.0,9595,Desk
6,McDonalds,0.0,0.0,9425,Desk
5,Lenovo,0.0,0.0,6136,Desk
3,Google,0.0,0.0,5931,Desk
1,Facebook,0.0,0.0,4896,Desk


In [62]:
top5

,EXPOSURE,MTM_T,MTM_T_1,PROFIT_LOSS,DeskName
4,Heineken,0.0,0.0,9595,Desk
6,McDonalds,0.0,0.0,9425,Desk
5,Lenovo,0.0,0.0,6136,Desk
3,Google,0.0,0.0,5931,Desk
1,Facebook,0.0,0.0,4896,Desk
